# Setup

In [1]:
### Libraries
import pandas as pd
from IPython.display import display
### Python OBDC bridge
import pyodbc 
### IRIS Python Native API
import irisnative

In [2]:
### SQL Connection parameters
dsn = 'IRIS IntegeratedML monitor'
server = 'irisimlsvr'
port = '51773'
database = 'USER' 
username = 'SUPERUSER' 
password = 'SYS' 
cnxn = pyodbc.connect('DRIVER={InterSystems ODBC35};SERVER='+server+';PORT='+port+';DATABASE='+database+';UID='+username+';PWD='+ password)

### charset stuffs...
cnxn.setdecoding(pyodbc.SQL_CHAR, encoding='utf8')
cnxn.setdecoding(pyodbc.SQL_WCHAR, encoding='utf8')
cnxn.setencoding(encoding='utf8')

### Get a SQL cursor
cursor = cnxn.cursor()

In [3]:
### Native API connection's parameters
ip = "irisimlsvr"
port = 51773
namespace = "USER"
username = "SUPERUSER"
password = "SYS"

### Create database connection and IRIS instance
try:
    nativeConn = irisnative.createConnection(ip, port, namespace, username, password)
    dbnative = irisnative.createIris(nativeConn)
except:
    print('Seems like you can\'t connect to IRIS... try to exit from any IRIS terminal or other license consumption session')

# ML model setup

In [4]:
### Model's parameters
dataTable = 'MedicalAppointments'
dataColumn = 'Show'
modelName = 'AppointmentsPredection'
trainLen = 500

In [5]:
### Clean previous runs
df1 = pd.read_sql(
    "SELECT COUNT(*) AS ModelExists FROM INFORMATION_SCHEMA.ML_TRAINED_MODELS WHERE MODEL_NAME = '%s'"
    % (modelName), 
    cnxn
)
if (df1['ModelExists'][0] == 1):
    print('Deleting previous model...')
    cursor.execute("DROP MODEL %s" % (modelName))
    print('Model deleted')
else:
    print('No previous runs found')

Deleting previous model...
Model deleted


In [6]:
### Create a model for predicting patient's appointments misses
### Note: seed parameter it's to force reproducibility
cursor.execute(
    "CREATE MODEL %s PREDICTING (%s) FROM %s USING {\"seed\": 3}" 
    % (modelName, dataColumn, dataTable)
)
print('Training model (this could take a while)...')
cursor.execute(
    "TRAIN MODEL %s FROM %s WHERE ID <= %s USING {\"seed\": 3}" 
    % (modelName, dataTable, trainLen)
)
print('Model trained')
cnxn.commit()

Training model (this could take a while)...
Model trained


In [7]:
### Display model information
cursor.execute("SELECT * FROM INFORMATION_SCHEMA.ML_TRAINED_MODELS")
df1 = pd.read_sql("SELECT * FROM INFORMATION_SCHEMA.ML_TRAINED_MODELS", cnxn)
display(df1)

,MODEL_NAME,TRAINED_MODEL_NAME,PROVIDER,TRAINED_TIMESTAMP,MODEL_TYPE,MODEL_INFO
0,AppointmentsPredection,AppointmentsPredection2,AutoML,2020-07-11 05:53:24.880,classification,"ModelType:Logistic Regression, Package:sklearn..."


In [8]:
### Display prediction and real patient's appointments presence
df1 = pd.read_sql(
    "SELECT PREDICT(%s) As Predicted, Show FROM %s WHERE ID <= %s" 
    % (modelName, dataTable, trainLen), 
    cnxn
)
display(df1)

,Predicted,Show
0,0,False
1,0,False
2,0,False
3,0,False
4,0,False
...,...,...
495,1,True
496,0,True
497,1,True
498,1,True


# Model monitoring

In [9]:
### Model validation
cursor.execute(
    "VALIDATE MODEL %s FROM %s WHERE ID < %s USING {\"seed\": 3}" 
    % (modelName, dataTable, 1000)
)
df5 = pd.read_sql(
    "SELECT * FROM INFORMATION_SCHEMA.ML_VALIDATION_METRICS WHERE MODEL_NAME = '%s'" 
    % (modelName), cnxn
)
df6 = df5.pivot(index='VALIDATION_RUN_NAME', columns='METRIC_NAME', values='METRIC_VALUE')
display(df6)

METRIC_NAME,Accuracy,F-Measure,Precision,Recall
VALIDATION_RUN_NAME,,,,
AppointmentsPredection21,0.83,0.89,0.95,0.83


As we can see, model's accuracy is about 83%.

MyMetric.IntegratedMLModelsValidation application monitor class let you to access such metrics. Thus it's possible to IRIS' ^%SYSMONMGR monitor utility being aware about your ML models perfomance.

In [10]:
### Get the last validation parameter for each IntegeratedML model
print(dbnative.classMethodValue("MyMetric.Install", "Test", "MyMetric.IntegratedMLModelsValidation"))

MyMetric - IntegratedMLModelsValidation   AppointmentsPredection26557121205331216   Serial value of object or the object identity:QMyMetric||IntegratedMLModelsValidation||AppointmentsPredection26557121205331216.MyMetric.Sample.IntegratedMLModelsValidation
   Accuracy:                   .83
   F-Measure:                  .89
   Precision:                  .95
   Recall:                     .83
   Name of the model definition:APPOINTMENTSPREDECTION
   Name of the trained model being validated:AppointmentsPredection26557121205331216
   Validation error (if encountered):1




Same results also could see by CSP interface: http://localhost:8092/csp/user/MyMetric.Sample.IntegratedMLModelsValidation.cls

With such functionality, someone could setup an alert into ^%SYSMONMGR monitor utility, checking model's performance metrics against thresholds. This way, IntegeratedML applications performance issues could be quickly notified.

For instance, let re-validate the model using first 5000 records and see how this relates to model's performance metrics.

In [11]:
### Model validation
cursor.execute(
    "VALIDATE MODEL %s FROM %s WHERE ID < %s USING {\"seed\": 3}" 
    % (modelName, dataTable, 5000)
)

In [12]:
### Get the last validation parameter for each IntegeratedML model
print(dbnative.classMethodValue("MyMetric.Install", "Test", "MyMetric.IntegratedMLModelsValidation"))

MyMetric - IntegratedMLModelsValidation   AppointmentsPredection26557121205693342   Serial value of object or the object identity:QMyMetric||IntegratedMLModelsValidation||AppointmentsPredection26557121205693342.MyMetric.Sample.IntegratedMLModelsValidation
   Accuracy:                   .71
   F-Measure:                  .82
   Precision:                  .82
   Recall:                     .82
   Name of the model definition:APPOINTMENTSPREDECTION
   Name of the trained model being validated:AppointmentsPredection26557121205693342
   Validation error (if encountered):1




Now model's accuracy had descreased to about 71%.  
If you had previously setup an alert to notify your team when model's performance is less than 80%, the team could be quickly notified and take care about this issue.

# Teardown

In [13]:
### Close the database and native API connections
nativeConn.close()
cnxn.close()